In [ ]:
import time
import sys
import os
path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)

from calibration.rfphasetune import RFPhaseTune


In [ ]:
# Initialize experiment
rfphtuner = RFPhaseTune()

# Optional: Adjust paraset
rfphtuner.paraset["rate_refresh"] = 10.0  # Hz
rfphtuner.set_stoptime(20)

# Setup experiment
rfphtuner.start()

In [ ]:
from IPython.display import display, clear_output

import plotly.graph_objs as go

# Setup live plot with Plotly
fig = go.FigureWidget()
fig.add_scatter(mode='lines+markers', name='Power vs Phase')
fig.update_layout(
    xaxis_title="Phase [deg]",
    yaxis_title="Power [uW]",
    title="Phase Tuning Live",
    xaxis=dict(range=[0, 360]),
    yaxis=dict(range=[0, 1.1 * rfphtuner.reference_power]),
    template="plotly_white"
)
display(fig)

# Run loop
while not rfphtuner.search_done and not rfphtuner.timeout():
    rfphtuner._run_exp()

    # Live update
    with fig.batch_update():
        fig.data[0].x = rfphtuner.dataset["phase_history"]
        fig.data[0].y = rfphtuner.dataset["power_history"]

# Shutdown
rfphtuner._shutdown_exp()

print("✅ Optimal phase:", rfphtuner.dataset["optimal_phase"])